In [1]:
# Import Libraries,
import pandas as pd
import re

In [2]:
# Read Data,
path = 'Data/flipkart.txt'
f = open(path, 'r', encoding='utf-8')
txt = f.read()

In [3]:
def fileReader(fileName, msgs, start=0):
    try:
        print("\n".join(fileName.split('\n')[start:msgs]))
    except:
        print("\n".join(fileName[start:msgs]))

fileReader(txt, 6)

09/10/21, 2:34 pm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.
09/10/21, 2:34 pm - This chat is with the official business account of Flipkart. Tap to learn more.
09/10/21, 2:34 pm - Flipkart: <Media omitted>
09/10/21, 2:34 pm - This chat is with the official business account of Flipkart. Tap to learn more.
12/11/21, 11:07 am - This business is now working with other companies to manage this chat. Tap to learn more.
18/12/21, 10:10 pm - Flipkart: <Media omitted>


In [4]:
# Seperate the messages and dates,
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[a-zA-Z]+\s-\s'
texts = re.split(pattern, txt)[1:]

In [5]:
fileReader(texts, 4)

Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.

This chat is with the official business account of Flipkart. Tap to learn more.

Flipkart: <Media omitted>

This chat is with the official business account of Flipkart. Tap to learn more.



In [6]:
dates = re.findall(pattern, txt)

In [7]:
fileReader(dates, 6)

09/10/21, 2:34 pm - 
09/10/21, 2:34 pm - 
09/10/21, 2:34 pm - 
09/10/21, 2:34 pm - 
12/11/21, 11:07 am - 
18/12/21, 10:10 pm - 


In [8]:
len(texts)

11

In [9]:
# Put everything into a data frame.
dic = {
    'messages' : texts,
    'date' : dates
}

df = pd.DataFrame(dic)
df.sample(3)

,messages,date
8,Flipkart: <Media omitted>\n,"12/06/22, 2:00 pm -"
9,This business is now working with other compan...,"19/11/23, 8:11 pm -"
7,Flipkart: Select what you want to buy: 👇Shoppi...,"20/12/21, 5:27 pm -"


In [10]:
# Change the date format to datetime format from a string,
df.date[0]

'09/10/21, 2:34\u202fpm - '

In [11]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y, %I:%M %p - ')
df.sample()

,messages,date
5,Flipkart: <Media omitted>\n,2021-12-18 22:10:00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   messages  11 non-null     object        
 1   date      11 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 304.0+ bytes


In [13]:
df.shape

(11, 2)

In [14]:
def dateConversion(df):

    df['Hour'] = df.date.dt.hour
    df['Minute'] = df.date.dt.minute
    df['Day'] = df.date.dt.day
    df['Month'] = df.date.dt.month_name()
    df['Year'] = df.date.dt.year

    return df.sample(5)

dateConversion(df)


,messages,date,Hour,Minute,Day,Month,Year
6,.: Menu\n,2021-12-20 17:27:00,17,27,20,December,2021
9,This business is now working with other compan...,2023-11-19 20:11:00,20,11,19,November,2023
2,Flipkart: <Media omitted>\n,2021-10-09 14:34:00,14,34,9,October,2021
5,Flipkart: <Media omitted>\n,2021-12-18 22:10:00,22,10,18,December,2021
0,Messages and calls are end-to-end encrypted. N...,2021-10-09 14:34:00,14,34,9,October,2021


In [15]:
def userSeperator(df):
    '''
    As the messages are in this format: {username}: {His/her message}
    Use regex to match and seperate user and message.
    '''
    usernames = []
    messages = []
    media = []
    pattern = '([\w\W]+?):\s'

    for chat in df['messages']:
        
        content = re.split(pattern, chat)

        if content[1:]:      # beginning from the first index as content contains blank ('') string at 0th index.
            usernames.append(content[1])
            pt = "<Media omitted>"
            st = content[2]
            msg = re.split(pt, st)

            if len(msg)==1:
                media.append(0)
                messages.append(msg[0])

            else:
                media.append(1)
                messages.append(msg[1])

        else:
            usernames.append('Zuckerberg')
            media.append(0)
            messages.append(content[0])
        
    df['message'] = messages
    df['user'] = usernames
    df['media'] = media
    
    return df.sample(5)

userSeperator(df)

,messages,date,Hour,Minute,Day,Month,Year,message,user,media
6,.: Menu\n,2021-12-20 17:27:00,17,27,20,December,2021,Menu\n,.,0
2,Flipkart: <Media omitted>\n,2021-10-09 14:34:00,14,34,9,October,2021,\n,Flipkart,1
0,Messages and calls are end-to-end encrypted. N...,2021-10-09 14:34:00,14,34,9,October,2021,Messages and calls are end-to-end encrypted. N...,Zuckerberg,0
10,This business is now working with other compan...,2023-12-10 01:16:00,1,16,10,December,2023,This business is now working with other compan...,Zuckerberg,0
8,Flipkart: <Media omitted>\n,2022-06-12 14:00:00,14,0,12,June,2022,\n,Flipkart,1


In [16]:
# Drop the unncessary columns,
df.drop(columns=['date','messages'], inplace=True)
df.sample(2)

,Hour,Minute,Day,Month,Year,message,user,media
10,1,16,10,December,2023,This business is now working with other compan...,Zuckerberg,0
4,11,7,12,November,2021,This business is now working with other compan...,Zuckerberg,0


__Other analysis__

In [19]:
from App.src import TextPreprocessor as tp

with open('Data/stopwords.txt') as f:
    stopwords = f.read()
    
new_df = df[df['user']!='Zuckerberg']
processed_msgs = []
for i in new_df['message'].values:
    html = tp.removeHtml(i)
    link = tp.removeUrl(html)
    punc = tp.removePunc(link)
    final_msg = punc.lower().split()

    for word in final_msg:
        if word not in stopwords:
            processed_msgs.append(word)

processed_msgs

['menu']

In [20]:
import pandas as pd
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import matplotlib
matplotlib.use('TkAgg')

text = new_df.message.str.cat(sep=' ')
wordcloud = WordCloud(width=500, 
                   height=500, 
                   scale=3, 
                   min_font_size=10, 
                   background_color="rgb(222, 245, 229)",
                   mode="RGB",
                   margin=0).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [10]:
with open('Data/dhruv.txt', 'r', encoding='utf-8') as f:
    df = f.read()

from App.src.Preprocessor import preprocess
df = preprocess(df)

In [34]:
a = df[df['user']!='Zuckerberg'].reset_index()

In [ ]:
new_df = df[df['user']!='Zuckerberg']
new_df = new_df[~new_df['message'].isin(['<Media omitted>\n', 'This message was deleted\n', 'You deleted this message\n'])]

In [6]:
from collections import Counter
final = text.split()

In [28]:
dict(Counter(final).most_common(10))

{'hai': 332,
 'nhi': 220,
 'to': 187,
 'bhi': 166,
 'h': 164,
 'hi': 161,
 'ke': 138,
 'kar': 121,
 'ki': 119,
 'me': 112}

In [2]:
time_df = df.groupby(['year', 'month_num', 'month']).count()['message'].reset_index()
time_df

,year,month_num,month,message
0,2021,8,August,35
1,2021,9,September,23
2,2021,10,October,56
3,2021,11,November,46
4,2021,12,December,85
5,2022,1,January,112
6,2022,2,February,139
7,2022,3,March,554
8,2022,4,April,341
9,2022,5,May,154


In [13]:
s = "hello, this is akshat, here's the contact number for the guy : Broker Vadodara.vcf (file attached) and 6 contacts.vcf (file attached)"

import re
def removeContacts(text):
    p = re.compile(r"[a-zA-Z0-9]* [A-z0-9]*\.vcf\s\(file attached\)")
    return p.sub(r'', text)

removeContacts(s)

"hello, this is akshat, here's the contact numbe for the guy :  and "